In [1]:
!pwd

/Users/zuck016/Projects/Proposals/PPI/CONCERTO/notebooks


In [2]:
import sys 
sys.path.append('../../RToru-GEM')

import rtoru
import cobra
import numpy as np

In [3]:
dir(rtoru)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'cobra',
 'exp_file_path',
 'expected_metab',
 'model',
 'path',
 'pathlib',
 'pd']

## Load models

In [4]:
from rtoru import model as rt

### Load reduced model

In [5]:
rt_reduced = cobra.io.read_sbml_model('../../RToruGEM/IFO0880_Reduced_Reactions_physically_derived_biomass.xml')
rt_reduced

Name,Rt_IFO0880
Memory address,108cd3eb0
Number of metabolites,2022
Number of reactions,2089
Number of genes,1137
Number of groups,0
Objective expression,1.0*R_BIOMASS - 1.0*R_BIOMASS_reverse_6575a
Compartments,"c, x, m, e, r, v, n, g, d"


In [6]:
rt.optimize()

,fluxes,reduced_costs
ALCD25yi,0.0,5.551115e-17
MTHFCm,0.0,1.536936e-17
AMPN,0.0,-3.772888e-03
DAGCPTer_RT,0.0,-2.220446e-16
PYRt2,0.0,0.000000e+00
...,...,...
EX_3cmp_e,0.0,0.000000e+00
EX_23cump_e,0.0,0.000000e+00
EX_thrp_e,0.0,0.000000e+00
EX_3ump_e,0.0,0.000000e+00


In [7]:
len(rt_reduced.reactions.R_BIOMASS.reactants)

143

In [8]:
print('\n'.join([f"{m.id}: {m.name}" for m in rt_reduced.reactions.R_BIOMASS.reactants]))

atp_c: ATP
gtp_c: GTP
gly_c: Glycine
ser__L_c: L-Serine
chtn_c: Chitin
chitos_c: Chitosan
acser_c: O-Acetyl-L-serine
cys__L_c: L-Cysteine
datp_c: DATP
ala__L_c: L-Alanine
gln__L_c: L-Glutamine
hdcea_e: Hexadecenoate (n-C16:1)
ocdcea_e: Octadecenoate (n-C18:1)
zymst_e: Zymosterol
pser__L_c: O-Phospho-L-serine
pro__L_m: L-Proline
ethamp_c: Ethanolamine phosphate
g3pc_e: Sn-Glycero-3-phosphocholine
pc_RT_e: Phosphatidylcholine
dctp_c: DCTP
sphgn_r: Sphinganine
glytrna_c: Glycyl-tRNA(Gly)
trnagly_c: TRNA(Gly)
ctp_c: CTP
pe_RT_r: Phosphatidylethanolamine
glutrna_c: L-Glutamyl-tRNA(Glu)
trnaglu_c: TRNA (Glu)
sph1p_r: Sphinganine 1-phosphate
his__L_c: L-Histidine
histrna_c: L-Histidyl-tRNA(His)
trnahis_c: TRNA(His)
phe__L_m: L-Phenylalanine
phetrna_m: L-Phenylalanyl-tRNA(Phe)
trnaphe_m: TRNA(Phe)
utp_c: UTP
leu__L_c: L-Leucine
thr__L_m: L-Threonine
cellb_c: Cellobiose
trnatrp_c: TRNA(Trp)
trp__L_c: L-Tryptophan
trptrna_c: L-Tryptophanyl-tRNA(Trp)
cyst__L_c: L-Cystathionine
suchms_c: O-Succiny

In [9]:
biomass = rt_reduced.reactions.get_by_any('R_BIOMASS')
rt_reduced.objective = 'R_BIOMASS'



rt.add_reactions(biomass)

rt.objective = 'R_BIOMASS'
biomass[0]

Reaction identifier,R_BIOMASS
Name,Biomass Objective
Memory address,0x139c99060
Stoichiometry,"30.1 13BDglcn_c + 7.2 2np6mep_m + 7.2 2np6mobq_m + 7.2 2npmhmobq_m + 8.4 3dsphgn_r + 7.2 3npab_m + 7.2 3npahb_m + 7.2 3npdhb_m + 7.2 3nphb_m + 8.4 4mzym_r + 7.2 8aonn_x + 145.0 acser_c + 6.0... 30.1 1 3 beta D Glucan C6H10O5 + 7.2 2-nonaprenyl-6-methoxyphenol + 7.2 2-nonaprenyl-6-methoxy-1,4-benzoquinone + 7.2 2-Decaprenyl-3-methyl-5-hydroxy-6-methoxy-1,4-benzoquinone + 8.4..."
GPR,
Lower bound,0.0
Upper bound,999999.0


In [10]:
for rxn in rt.reactions:
    try:
        rt_reduced.reactions.get_by_id(rxn.id)
    except KeyError:
        rxn.objective_coefficient = -1
rt.reactions.BIOMASS_RT.objective_coefficient = 1000

In [11]:
sol = rt.optimize()
fluxes = sol.fluxes.to_frame('flux').sort_values('flux')
nonzero_fluxes = fluxes[~np.isclose(fluxes, 0)]
nonzero_fluxes

,flux
H2Otm,-14.679891
H2Ot,-2.809183
CO2tm,-2.412514
EX_pi_e,-2.316568
EX_o2_e,-2.287993
...,...
NADH2_u9m2,3.257166
CYOR_u9m,4.124446
ATPS3m,12.471465
PIt2m,12.852601


In [12]:
nonzero_fluxes.loc['BIOMASS_RT']

flux    0.073353
Name: BIOMASS_RT, dtype: float64

In [13]:
sol

,fluxes,reduced_costs
ALCD25yi,0.0,3.655016e-13
MTHFCm,0.0,-1.587737e-16
AMPN,0.0,-3.921961e+00
DAGCPTer_RT,0.0,0.000000e+00
PYRt2,0.0,2.220446e-16
...,...,...
EX_23cump_e,0.0,-0.000000e+00
EX_thrp_e,0.0,-0.000000e+00
EX_3ump_e,0.0,-0.000000e+00
EX_tyrp_e,0.0,-0.000000e+00


In [14]:
active_unmapped = [rxn for rxn in nonzero_fluxes.index 
                   if rt.reactions.get_by_id(rxn).objective_coefficient == -1]
nonzero_fluxes.loc[active_unmapped]
    

,flux
EX_pi_e,-2.316568
EX_o2_e,-2.287993
EX_glc__D_e,-1.000000
EX_nh4_e,-0.484152
EX_k_e,-0.042963
EX_so4_e,-0.007353
EX_mg2_e,-0.004527
EX_na1_e,-0.001914
EX_fe3_e,-0.000453
EX_ca2_e,-0.000061


In [15]:
len(active_unmapped)

49

In [ ]:
len

In [17]:
dir(biomass)

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [16]:
rt_reduced.reactions.get_by_id('RT_BIOMASS')

KeyError: 'RT_BIOMASS'